# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Procesamiento de Lenguaje Natural**

### Tecnológico de Monterrey

### Prof Luis Eduardo Falcón Morales

## Adtividad Semana 5

### **Vectores Embebidos de OpenAI**

#### **Nombres y matrículas de los integrantes del equipo:**



*   Arturo Jain Delgadillo (A01794992)
*   Ramiro Martin Jaramillo Romero (A01171251)
*   José Ashamat Jaimes Saavedra (A01736690)
*   Owen Jáuregui Borbón (A01638122)



In [ ]:
# Aquí deberás incluir todas las librerías que requieras durante esta actividad:
import os, json, pickle, re, string, unicodedata
from pathlib import Path
from collections import Counter

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline        import make_pipeline
from sklearn.preprocessing   import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

import time, math, openai, tiktoken
from tqdm.auto import tqdm

import nltk
from nltk.corpus import stopwords

# Rutas
DATA_DIR  = Path("/content/drive/MyDrive/2025-1/Vectores Embebidos/")
amazon_p  = DATA_DIR / "amazon5.txt"
imdb_p    = DATA_DIR / "imdb5.txt"
yelp_p    = DATA_DIR / "yelp5.txt"
CACHE_DIR = Path("cache"); CACHE_DIR.mkdir(exist_ok=True)

In [ ]:
# Incluye las celdas necesarias para tu acceso a la API de OpenAI.

# API de OpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "<Poner aquí tu API KEY>")
import openai
openai.api_key = OPENAI_API_KEY

# **Pregunta - 1:**



Descarga los 3 archivos de Canvas y genera un nuevo DataFrame de Pandas con ellos.

**Llama simplemente "df" a dicho DataFrame.**

Los archivos los encuentras en Canvas: amazon5.txt, imdb5.txt, yelp5.txt.



In [ ]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

def load_txt(path: Path) -> pd.DataFrame:
    """Lee archivos donde cada línea termina con la etiqueta 0/1.

    El separador puede ser TAB (`\t`) –caso Amazon/Yelp– o varios espacios –caso IMDB–.
    Ignora líneas vacías o cabeceras que no terminen con 0 o 1.
    """
    rows = []
    with open(path, encoding="utf-8") as f:
        for line in f:
            line = line.rstrip("\n")
            if not line.strip():
                continue  # salta líneas en blanco
            # Intenta primero con TAB; si no, con espacios múltiples
            if "\t" in line:
                parts = line.rsplit("\t", 1)
            else:
                parts = line.rsplit(" ", 1)
            if len(parts) != 2 or parts[1] not in {"0", "1"}:
                # línea malformada (e.g. cabecera); se omite
                continue
            txt, lab = parts[0].strip(), parts[1]
            rows.append({"text": txt, "label": int(lab)})
    if not rows:
        raise ValueError(f"No se pudieron parsear registros válidos en {path}")
    return pd.DataFrame(rows)

# Carga y concatenación
frames = [load_txt(p) for p in (amazon_p, imdb_p, yelp_p)]
df = pd.concat(frames, ignore_index=True)
print("Shape final:", df.shape)  # esperado: (3000, 2)

# *********** Aquí termina la sección de agregar código *************


Shape final: (3000, 2)


In [ ]:
# Verifiquemos la información del DataFrame:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    3000 non-null   object
 1   label   3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


In [ ]:
# Y veamos sus primeros registros:

df.head()

,text,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


# **Pregunta - 2:**

Realiza el proceso de limpieza. Aplica el preprocesamiento que consideres adecuado.











In [ ]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


nltk.download("stopwords", quiet=True)
STOP = set(stopwords.words("english"))
_url_pat = re.compile(r"http\S+|www\S+")

def clean(text: str) -> str:
    text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode()
    text = text.lower()
    text = _url_pat.sub("", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    toks = [w for w in text.split() if w not in STOP]
    return " ".join(toks)

# Xclean y Y
Xclean = df["text"].map(clean).values
Y      = df["label"].values


# *********** Aquí termina la sección de agregar código *************

In [ ]:
# Despleguemos los primeros comentarios después de tu proceso de limpieza:

for x in Xclean[0:5]:
  print(x)


way plug us unless go converter
good case excellent value
great jawbone
tied charger conversations lasting 45 minutesmajor problems
mic great


# **Pregunta - 3:**



Realicemos una partición aleatoria con los mismos porcentajes de la práctica pasada para poder comparar dichos resultados con los de
esta actividad, a saber, 70%, 15% y 15%, para entrenamiento, validación y prueba, respectivamente.

In [ ]:

# ************* Inicia la sección de agregar código:*****************************

Xclean_train, Xclean_temp, Y_train, Y_temp = train_test_split(
    Xclean, Y, test_size=0.30, random_state=42, stratify=Y)
Xclean_val, Xclean_test, Y_val, Y_test = train_test_split(
    Xclean_temp, Y_temp, test_size=0.50, random_state=42, stratify=Y_temp)

x_train, x_val, x_test = Xclean_train, Xclean_val, Xclean_test
y_train, y_val, y_test = Y_train, Y_val, Y_test

# *********** Termina la sección de agregar código *************


# verificemos las dimensiones obtenidas:
print('X,y Train:', len(x_train), len(y_train))
print('X,y Val:', len(x_val), len(y_val))
print('X,y Test', len(x_test), len(y_test))

X,y Train: 2100 2100
X,y Val: 450 450
X,y Test 450 450


# **Pregunta - 4:**



Construye tu vocabulario a continuación


In [ ]:
# a.	Usa el conjunto de entrenamiento para generar tu vocabulario
#     con un tamaño que consideres adecuado:


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


min_freq, min_len = 2, 3  # ajusta si quieres experimentar
cnt = Counter(tok for doc in Xclean_train for tok in doc.split())
vocab = {tok for tok, c in cnt.items() if c >= min_freq and len(tok) >= min_len}


# *********** Aquí termina la sección de agregar código *************

In [ ]:
# b.	Indica el tamaño del vocabulario generado.

print('Longitud del vocabulario generado:')


# ******* Inicia la sección de agregar código: ***********


len(vocab)


# *********** Aquí termina la sección de agregar código *************

Longitud del vocabulario generado:


1595

c.	¿Por qué debe usarse solamente el conjunto de entrenamiento para generar el vocabulario?


### ++++++++ Inicia la sección de agregar texto: +++++++++++

El vocabulario se construye SOLO con Train para evitar data leakage.

### ++++++++ Termina la sección de agregar texto: +++++++++++


In [ ]:
# d.	Con el vocabulario generado, filtra los conjuntos de entrenamiento,
#     validación y prueba para que todos los comentarios usen solamente las
#     palabras de este vocabulario.

#     Llamar train_x, val_x y test_x a estos tres conjuntos.


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


def filter_doc(doc: str) -> str:
    return " ".join(tok for tok in doc.split() if tok in vocab)

train_docs = [filter_doc(d) for d in Xclean_train]
val_docs   = [filter_doc(d) for d in Xclean_val]
test_docs  = [filter_doc(d) for d in Xclean_test]

train_x, val_x, test_x = train_docs, val_docs, test_docs


# *********** Aquí termina la sección de agregar código *************


In [ ]:
# Vemos el resultado de los primeros comentarios del conjunto de entrenamiento:

for ss in train_x[0:5]:
  print(ss)

least passed ordering food wasnt busy
extraordinary liked sliced white
use french three films nothing short incredible every shot every scene like work art
predictable bad watch
enjoyed


# **Pregunta - 5:**


#### **Incluye aquí un resumen de las características y diferencias que tiene al menos los tres modelos de OpenAI indicados: "text-embedding-3-small", "text-embedding-3-large" y "text-embedding-ada-002".**

### ++++++++ Inicia la sección de agregar texto: +++++++++++

`text-embedding-3-small`

* Dimensiones: 1,536
* Ventana de contexto: hasta 8,192 tokens
* Costo: moderado
* Ideal para: proyectos que buscan buen rendimiento sin gastar mucho.

`text-embedding-3-large`

* Dimensiones: 3,072
* Ventana de contexto: hasta 8,192 tokens
* Costo: más alto
* Ideal para: aplicaciones que requieren la máxima precisión.

`text-embedding-ada-002`

* Dimensiones: 1,536
* Ventana de contexto: hasta 8,192 tokens
* Costo: muy bajo
* Ideal para: proyectos con presupuesto limitado o tareas básicas.

### ++++++++ Termina la sección de agregar texto: +++++++++++


# **Pregunta - 6:**


#### **Diccionario clave-valor de palabras del diccionario y vectores embebidos.**

In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


EMB_MODEL   = "text-embedding-3-small"
CACHE_FILE  = CACHE_DIR / "emb_vocab.pkl"
BATCH_SIZE  = 100
enc = tiktoken.encoding_for_model(EMB_MODEL)

# ---------- 1. Carga caché si existe -----------------------------------------
if CACHE_FILE.exists():
    with open(CACHE_FILE, "rb") as f:
        emb_vocab, total_tokens = pickle.load(f)
    print(f"⬇️  Cargados {len(emb_vocab):,} embeddings de {CACHE_FILE}")
else:
    emb_vocab, total_tokens = {}, 0

# ---------- 2. Embeddings faltantes ------------------------------------------
missing = sorted([w for w in vocab if w not in emb_vocab])
print("🔍 Palabras faltantes:", len(missing))

for i in tqdm(range(0, len(missing), BATCH_SIZE), desc="Solicitando a OpenAI"):
    batch = missing[i:i+BATCH_SIZE]

    # tokens facturados para este batch
    total_tokens += sum(len(enc.encode(w)) for w in batch)

    response = openai.embeddings.create(model=EMB_MODEL, input=batch)
    for obj in response.data:
        # Access attributes using dot notation instead of square brackets
        emb_vocab[batch[obj.index]] = obj.embedding

    time.sleep(0.3)     # evita rate-limit

# ---------- 3. Guarda diccionario + metadatos --------------------------------
with open(CACHE_FILE, "wb") as f:
    pickle.dump((emb_vocab, total_tokens), f)

print(f"💾 Diccionario guardado en: {CACHE_FILE}")
print(f"📐 Dimensión del vector: {len(next(iter(emb_vocab.values())))}")
print(f"🔢 Tokens facturados acumulados: {total_tokens:,}")

# ---------- 4. Estimación rápida de costo (USD) ------------------------------
PRICE_PER_1K = {"text-embedding-3-small": 0.00002,
                "text-embedding-3-large": 0.00006,
                "text-embedding-ada-002": 0.00010}
usd_cost = total_tokens/1000 * PRICE_PER_1K[EMB_MODEL]
print(f"💲 Costo aproximado: ${usd_cost:.4f} USD")


# *********** Aquí termina la sección de agregar código *************

🔍 Palabras faltantes: 1595


Solicitando a OpenAI:   0%|          | 0/16 [00:00<?, ?it/s]

💾 Diccionario guardado en: cache/emb_vocab.pkl
📐 Dimensión del vector: 1536
🔢 Tokens facturados acumulados: 2,413
💲 Costo aproximado: $0.0000 USD


# **Pregunta - 7:**



Generamos los vectores embebidos a partir de los conjuntos de entrenamiento, validación y prueba.

Los llamaremos trainEmb, valEmb y testEmb, respectivamente.

In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


DIM = len(next(iter(emb_vocab.values())))

def doc2vec(doc: str) -> np.ndarray:
    vecs = [emb_vocab[w] for w in doc.split() if w in emb_vocab]
    if not vecs:
        return np.zeros(DIM, dtype="float32")
    return np.mean(vecs, axis=0).astype("float32")

def build_matrix(docs, name):
    out = CACHE_DIR / f"{name}.npy"
    if out.exists():
        return np.load(out)
    mat = np.vstack([doc2vec(d) for d in docs])
    np.save(out, mat)
    return mat

trainEmb = build_matrix(train_x, "trainEmb")
valEmb   = build_matrix(val_x,   "valEmb")
testEmb  = build_matrix(test_x,  "testEmb")

# % de comentarios sin ninguna palabra reconocida
pct_empty = 100 * (trainEmb == 0).all(axis=1).mean()
print(f"Comentarios sin tokens del vocabulario: {pct_empty:.2f}%")


# *********** Aquí termina la sección de agregar código *************

Comentarios sin tokens del vocabulario: 0.67%


In [ ]:
# Veamos las dimensiones de cada conjunto embebido:

print("Train-Emb:", trainEmb.shape)
print("Val-Emb:", valEmb.shape)
print("Test-Emb:", testEmb.shape)

Train-Emb: (2100, 1536)
Val-Emb: (450, 1536)
Test-Emb: (450, 1536)


# **Pregunta - 8:**



Utiliza los modelos de regresión logística y bosque aleatorio (random forest) y encuentra sus desempeños.

Compara los resultados con los de la semana anterior.

In [ ]:
# REGRESIÓN LOGÍSTICA:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


logreg = make_pipeline(
    StandardScaler(with_mean=False),        # no centra (matrices dispersas)
    LogisticRegression(
        penalty="l2",
        max_iter=1000,
        random_state=42,
        n_jobs=-1,
    )
).fit(trainEmb, Y_train)

Y_val_pred_lr = logreg.predict(valEmb)
val_acc_lr    = accuracy_score(Y_val, Y_val_pred_lr)

print("=== Regresión Logística – VALIDACIÓN ===")
print(f"Accuracy: {val_acc_lr:.4f}")
print(classification_report(Y_val, Y_val_pred_lr, digits=4))


# *********** Aquí termina la sección de agregar código *************


=== Regresión Logística – VALIDACIÓN ===
Accuracy: 0.7222
              precision    recall  f1-score   support

           0     0.7451    0.6756    0.7086       225
           1     0.7033    0.7689    0.7346       225

    accuracy                         0.7222       450
   macro avg     0.7242    0.7222    0.7216       450
weighted avg     0.7242    0.7222    0.7216       450



In [ ]:
# BOSQUE ALEATORIO (Random Forest):

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


rf = RandomForestClassifier(
        n_estimators=500,
        max_depth=None,
        random_state=42,
        n_jobs=-1,
    ).fit(trainEmb, Y_train)

Y_val_pred_rf = rf.predict(valEmb)
val_acc_rf    = accuracy_score(Y_val, Y_val_pred_rf)

print("\n=== Random Forest – VALIDACIÓN ===")
print(f"Accuracy: {val_acc_rf:.4f}")
print(classification_report(Y_val, Y_val_pred_rf, digits=4))


# *********** Aquí termina la sección de agregar código *************


=== Random Forest – VALIDACIÓN ===
Accuracy: 0.7800
              precision    recall  f1-score   support

           0     0.8214    0.7156    0.7648       225
           1     0.7480    0.8444    0.7933       225

    accuracy                         0.7800       450
   macro avg     0.7847    0.7800    0.7791       450
weighted avg     0.7847    0.7800    0.7791       450



# **Pregunta - 9:**



Reporte del mejor modelo con el conjunto de Prueba (Test).


In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


best_model, best_name, best_val_acc = (logreg, "LogReg", val_acc_lr) \
    if val_acc_lr >= val_acc_rf else (rf, "RandForest", val_acc_rf)

print(f"\n🏆  Mejor modelo en validación: {best_name} "
      f"(accuracy {best_val_acc:.4f})")

Y_test_pred = best_model.predict(testEmb)
print("=== Desempeño en TEST ===")
print(classification_report(Y_test, Y_test_pred, digits=4))


# *********** Aquí termina la sección de agregar código *************


🏆  Mejor modelo en validación: RandForest (accuracy 0.7800)
=== Desempeño en TEST ===
              precision    recall  f1-score   support

           0     0.8111    0.7822    0.7964       225
           1     0.7897    0.8178    0.8035       225

    accuracy                         0.8000       450
   macro avg     0.8004    0.8000    0.7999       450
weighted avg     0.8004    0.8000    0.7999       450



# **Pregunta - 10:**

In [ ]:
# Incluye todas las líneas de código y celdas que consideres adecuadas para este ejercicio.


# ---------- a) Embeddings de los 3 000 comentarios --------------------------
DOC_MODEL  = "text-embedding-3-small"
DOC_FILE   = CACHE_DIR / "docEmbeddings.npy"   # matriz (3000, dim)
TOK_FILE   = CACHE_DIR / "doc_tokens.pkl"      # cache de tokens usados

enc = tiktoken.encoding_for_model(DOC_MODEL)
if openai.api_key is None:                      # ya se configuró antes
    openai.api_key = os.getenv("OPENAI_API_KEY")

if DOC_FILE.exists() and TOK_FILE.exists():
    fullEmb   = np.load(DOC_FILE)
    total_tok = pickle.loads(TOK_FILE.read_bytes())
    print(f"⬇️  Embeddings cargados de {DOC_FILE}")
else:
    comments   = df["text"].tolist()            # ← texto original
    dims       = None
    fullEmb    = np.empty((len(comments), 1536), dtype="float32")  # 1536 para 3-small
    total_tok  = 0

    for i in tqdm(range(len(comments)), desc="OpenAI – doc embeddings"):
        txt   = comments[i]
        total_tok += len(enc.encode(txt))
        rsp   = openai.embeddings.create(model=DOC_MODEL, input=txt)
        vec   = rsp.data[0].embedding
        if dims is None:                        # 1ª vez sabemos la dimensión real
            dims = len(vec)
            fullEmb = np.empty((len(comments), dims), dtype="float32")
        fullEmb[i] = vec
        time.sleep(0.15)                        # limita QPS

    np.save(DOC_FILE, fullEmb)
    TOK_FILE.write_bytes(pickle.dumps(total_tok))
    print(f"💾  Matriz guardada en {DOC_FILE}")

print(f"🔢  Tokens facturados (comentarios): {total_tok:,}")
usd = total_tok/1000 * 0.00002                 # precio text-embedding-3-small
print(f"💲  Costo aprox.: ${usd:.4f} USD")

# ---------- b) Split 70-15-15 con la misma semilla 42 -----------------------
X_train, X_temp, Y_train_b, Y_temp_b = train_test_split(
    fullEmb, Y, test_size=0.30, random_state=42, stratify=Y)
X_val,   X_test, Y_val_b,   Y_test_b = train_test_split(
    X_temp,  Y_temp_b, test_size=0.50, random_state=42, stratify=Y_temp_b)

print("Shapes:", X_train.shape, X_val.shape, X_test.shape)

# ---------- c-1) Regresión Logística ----------------------------------------
pipe_lr = make_pipeline(
    StandardScaler(with_mean=False),
    LogisticRegression(max_iter=1000, n_jobs=-1, random_state=42)
).fit(X_train, Y_train_b)

print("\n=== LogisticRegression – VALIDACIÓN ===")
print(classification_report(Y_val_b, pipe_lr.predict(X_val), digits=4))

# ---------- c-2) Random Forest ----------------------------------------------
rf_b = RandomForestClassifier(
    n_estimators=500, random_state=42, n_jobs=-1).fit(X_train, Y_train_b)

print("\n=== Random Forest – VALIDACIÓN ===")
print(classification_report(Y_val_b, rf_b.predict(X_val), digits=4))

# ---------- c-3) Desempeño final en TEST + comparación ----------------------
for name, mdl in [("LogReg", pipe_lr), ("RandForest", rf_b)]:
    print(f"\n🏁  {name} – TEST")
    print(classification_report(Y_test_b, mdl.predict(X_test), digits=4))




OpenAI – doc embeddings:   0%|          | 0/3000 [00:00<?, ?it/s]

💾  Matriz guardada en cache/docEmbeddings.npy
🔢  Tokens facturados (comentarios): 43,804
💲  Costo aprox.: $0.0009 USD
Shapes: (2100, 1536) (450, 1536) (450, 1536)

=== LogisticRegression – VALIDACIÓN ===
              precision    recall  f1-score   support

           0     0.9736    0.9822    0.9779       225
           1     0.9821    0.9733    0.9777       225

    accuracy                         0.9778       450
   macro avg     0.9778    0.9778    0.9778       450
weighted avg     0.9778    0.9778    0.9778       450


=== Random Forest – VALIDACIÓN ===
              precision    recall  f1-score   support

           0     0.9865    0.9778    0.9821       225
           1     0.9780    0.9867    0.9823       225

    accuracy                         0.9822       450
   macro avg     0.9823    0.9822    0.9822       450
weighted avg     0.9823    0.9822    0.9822       450


🏁  LogReg – TEST
              precision    recall  f1-score   support

           0     0.9822    0.9822

# **Pregunta - 11:**



Incluye tus comentarios finales de la actividad.

### ++++++++ Inicia la sección de agregar texto: +++++++++++

Esta actividad nos permitió comprender a fondo cómo los modelos de embeddings de OpenAI pueden utilizarse para representar texto de manera densa y significativa en tareas de procesamiento de lenguaje natural. Al trabajar con distintos modelos como text-embedding-3-small y text-embedding-ada-002, observamos cómo varía el desempeño del modelo de clasificación en función del tipo de embedding utilizado, lo cual es clave al momento de elegir un modelo según los recursos y objetivos de un proyecto.

Además, nos pareció especialmente valioso integrar los embeddings con modelos clásicos como regresión logística y random forest. Esto reafirma que técnicas avanzadas de representación de texto pueden aprovecharse incluso con clasificadores tradicionales, siempre que se haga un buen preprocesamiento y selección de características.

### ++++++++ Termina la sección de agregar texto: +++++++++++

# **Fin de la Actividad de Vectores Embebidos - OpenAI**